In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [5]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [6]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train= sc.fit_transform(x_train)
x_test= sc.transform(x_test)
#  (Xi-Xmean)/Xstd

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid= {'criterion':['squared_error','friedman_mse','absolute_error','poisson'],'max_features': ['sqrt','log2'],'n_estimators':[50,100]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\santhiya\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             verbose=3)

In [10]:
re=grid.cv_results_
re

{'mean_fit_time': array([0.15978966, 0.28311076, 0.15180793, 0.29688988, 0.16145411,
        0.28725748, 0.16087122, 0.29467421, 0.45812645, 0.94345593,
        0.51699281, 0.94172773, 0.19901328, 0.35227027, 0.1778903 ,
        0.2861021 ]),
 'std_fit_time': array([0.01028652, 0.01614414, 0.01164265, 0.0080368 , 0.01564077,
        0.01730566, 0.01100585, 0.04403734, 0.02552894, 0.08713533,
        0.04006997, 0.06986246, 0.02110189, 0.01009587, 0.01265375,
        0.03236001]),
 'mean_score_time': array([0.00974698, 0.01104379, 0.00966458, 0.01147175, 0.00421467,
        0.01007867, 0.00817175, 0.01203642, 0.00873194, 0.01045051,
        0.00681925, 0.01003532, 0.00976863, 0.01000538, 0.00441275,
        0.00643907]),
 'std_score_time': array([6.38936780e-03, 4.03873198e-03, 8.22175184e-04, 3.06099300e-03,
        4.58067659e-03, 4.97007532e-05, 7.17146893e-03, 3.11054032e-03,
        1.77524768e-03, 8.05660591e-04, 3.92830395e-03, 1.10121513e-05,
        1.10672768e-02, 1.28482491e-

In [11]:
grid_predictions=grid.predict(x_test)

In [12]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.159790,0.010287,0.009747,0.006389,squared_error,sqrt,50,"{'criterion': 'squared_error', 'max_features':...",0.861836,0.793138,0.809412,0.833133,0.760268,0.811557,0.034548,4
1,0.283111,0.016144,0.011044,0.004039,squared_error,sqrt,100,"{'criterion': 'squared_error', 'max_features':...",0.865284,0.781040,0.805350,0.828482,0.771712,0.810373,0.033833,9
2,0.151808,0.011643,0.009665,0.000822,squared_error,log2,50,"{'criterion': 'squared_error', 'max_features':...",0.867578,0.788210,0.804168,0.829480,0.763162,0.810519,0.035766,8
3,0.296890,0.008037,0.011472,0.003061,squared_error,log2,100,"{'criterion': 'squared_error', 'max_features':...",0.862853,0.789603,0.805141,0.825039,0.766142,0.809756,0.032810,12
4,0.161454,0.015641,0.004215,0.004581,friedman_mse,sqrt,50,"{'criterion': 'friedman_mse', 'max_features': ...",0.865991,0.785007,0.800474,0.821630,0.772505,0.809121,0.032826,14
5,0.287257,0.017306,0.010079,0.000050,friedman_mse,sqrt,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.862105,0.786765,0.804857,0.826201,0.766054,0.809196,0.033081,13
6,0.160871,0.011006,0.008172,0.007171,friedman_mse,log2,50,"{'criterion': 'friedman_mse', 'max_features': ...",0.857061,0.783589,0.808917,0.835357,0.764966,0.809978,0.033420,10
7,0.294674,0.044037,0.012036,0.003111,friedman_mse,log2,100,"{'criterion': 'friedman_mse', 'max_features': ...",0.865274,0.793498,0.807350,0.827196,0.775850,0.813834,0.030735,2
8,0.458126,0.025529,0.008732,0.001775,absolute_error,sqrt,50,"{'criterion': 'absolute_error', 'max_features'...",0.853355,0.787884,0.807369,0.825205,0.765679,0.807898,0.030163,15
9,0.943456,0.087135,0.010451,0.000806,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.859572,0.797576,0.814660,0.825668,0.768211,0.813137,0.030252,3


In [13]:
from sklearn.metrics import r2_score
r_score = r2_score(y_test,grid_predictions)
print(r_score)
print("The R_score value for best parameter {}:".format(grid.best_params_))

0.868816211329877
The R_score value for best parameter {'criterion': 'poisson', 'max_features': 'log2', 'n_estimators': 50}:


In [ ]:
#age_input=float(input("Age:"))
#bmi_input=float(input("BMI:"))
#children_input=float(input("Children:"))
#sex_male_input=int(input("Sex Male 0 or 1:"))
#smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

In [ ]:
#Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])

#print("Future_Prediction={}".format(Future_Prediction))
